In [11]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import pickle

from scipy.spatial.distance import cdist
from torch_geometric.utils import from_scipy_sparse_matrix
from scipy.sparse import coo_matrix

In [12]:
reservoir_info = {
    "BSR": 1990,
    "CAU": 1999,
    "CRY": 1982,
    "DCR": 1987,
    "DIL": 1985,
    "ECH": 1982,
    "ECR": 1992,
    "FGR": 1982,
    "FON": 1990,
    "GMR": 1982,
    "HYR": 1999,
    "JOR": 1997,
    "JVR": 1996,
    "LCR": 1998,
    "LEM": 1982,
    "MCP": 1991,
    "MCR": 1998,
    "NAV": 1986,
    "PIN": 1990,
    "RFR": 1989,
    "RID": 1990,
    "ROC": 1982,
    "RUE": 1982,
    "SCO": 1996,
    "SJR": 1992,
    "STA": 1982,
    "STE": 1982,
    "TPR": 1982,
    "USR": 1991,
    "VAL": 1986,
}

In [13]:
rsrs = reservoir_info.keys()
map = gpd.read_file("./data/map/ReservoirElevations.shp")
map = map[['Initials', 'Lat', 'Lon', 'RASTERVALU']]
map.loc[map['Initials'] == 'TAY', 'Initials'] = 'TPR'
map = map[map['Initials'].isin(rsrs)]
map.reset_index(drop=True, inplace=True)
map.columns = ['RSR', 'LAT', 'LON', 'ELEV']

In [14]:
def build_rsr_graph(df, _nearest_k=2, _elevation=True, _post_choice=False, _self_edges=0):
    """Construct a graph adjacency matrix for reservoirs based on distance and elevation.
    Args:
        df: DataFrame with columns ['RSR', 'LAT', 'LON', 'ELEV']
        _nearest_k: Number of nearest neighbors to consider
        _elevation: Whether to consider elevation constraints
        _post_choice: If True, filter by elevation after selecting k-nearest;
                     If False, filter by elevation before selecting k-nearest
        _self_edges: Value to add to diagonal (self-edges) of adjacency matrix
    Returns:
        tuple: (A, encode_map, num_edges) where:
            - A: adjacency matrix (n x n)
            - encode_map: dictionary mapping reservoir name to index
            - num_edges: total number of valid edges
    """
    n = len(df)
    rsrs = df['RSR'].values
    coords = df[['LAT', 'LON']].values
    elevs = df['ELEV'].values
    encode_map = {rsr: i for i, rsr in enumerate(rsrs)} # Create encoding map
    A = np.zeros((n, n), dtype=int)  # Initialize adjacency matrix
    dist_matrix = cdist(coords, coords, metric='euclidean')

    for i in range(n):
        current_elev = elevs[i]
        distances = dist_matrix[i]
        if _elevation and not _post_choice:
            # Pre-filter: only consider reservoirs with lower elevation
            valid_candidates = []
            for j in range(n):
                if i != j and elevs[j] < current_elev:
                    valid_candidates.append((distances[j], j))
            # Sort by distance and take k nearest
            valid_candidates.sort(key=lambda x: x[0])
            selected_neighbors = [idx for _, idx in valid_candidates[:_nearest_k]]
        else:
            # Find k nearest neighbors first (excluding self)
            neighbor_indices = np.argsort(distances)
            # Exclude self (index i)
            neighbor_indices = neighbor_indices[neighbor_indices != i]
            selected_neighbors = neighbor_indices[:_nearest_k].tolist()
            if _elevation and _post_choice:
                # Post-filter: remove neighbors with higher elevation
                selected_neighbors = [j for j in selected_neighbors if elevs[j] < current_elev]

        for j in selected_neighbors:
            A[i, j] = 1

    # Add self-edges
    if _self_edges != 0:
        np.fill_diagonal(A, _self_edges)

    num_edges = np.sum(A)

    return A, encode_map, num_edges

In [15]:
os.makedirs('data/graph', exist_ok=True)

print("==" * 50)
print("Testing adjacency matrix construction:")

# Configuration 0: Self-loops only (k=0)
A0, encode_map0, num_edges0 = build_rsr_graph(map, _nearest_k=0, _elevation=False, _self_edges=1)
edge_index0, _ = from_scipy_sparse_matrix(coo_matrix(A0))
edge_index0 = edge_index0.long()
config0_data = {
    'A': A0,
    'encode_map': encode_map0,
    'edge_index': edge_index0,
}
with open('data/graph/config0.pkl', 'wb') as f:
    pickle.dump(config0_data, f)
print(f"Config 0 saved: {A0.shape} adjacency matrix with {num_edges0} edges, {edge_index0.shape} edges in edge_index")


for _nearest_k in range(2, 16):  # Orginal setting: 2

    print(f"\nBuilding graph with k={_nearest_k}")
    # Configuration 1: no elevation constraint
    A1, encode_map1, num_edges1 = build_rsr_graph(map, _nearest_k=_nearest_k, _elevation=False)
    edge_index1, _ = from_scipy_sparse_matrix(coo_matrix(A1))
    edge_index1 = edge_index1.long()
    config1_data = {
        'A': A1,
        'encode_map': encode_map1,
        'edge_index': edge_index1,
    }
    with open(f'data/graph/k{_nearest_k}_config1.pkl', 'wb') as f:
        pickle.dump(config1_data, f)
    print(f"Config 1 saved: {A1.shape} adjacency matrix with {num_edges1} edges, {edge_index1.shape} edges in edge_index")

    # Configuration 2: elevation constraint (pre-filter)
    A2, encode_map2, num_edges2 = build_rsr_graph(map, _nearest_k=_nearest_k, _elevation=True, _post_choice=False)
    edge_index2, _ = from_scipy_sparse_matrix(coo_matrix(A2))
    edge_index2 = edge_index2.long()
    config2_data = {
        'A': A2,
        'encode_map': encode_map2,
        'edge_index': edge_index2,
    }
    with open(f'data/graph/k{_nearest_k}_config2.pkl', 'wb') as f:
        pickle.dump(config2_data, f)
    print(f"Config 2 saved: {A2.shape} adjacency matrix with {num_edges2} edges, {edge_index2.shape} edges in edge_index")

    # Configuration 3: elevation constraint (post-filter)
    A3, encode_map3, num_edges3 = build_rsr_graph(map, _nearest_k=_nearest_k, _elevation=True, _post_choice=True)
    edge_index3, _ = from_scipy_sparse_matrix(coo_matrix(A3))
    edge_index3 = edge_index3.long()
    config3_data = {
        'A': A3,
        'encode_map': encode_map3,
        'edge_index': edge_index3,
    }
    with open(f'data/graph/k{_nearest_k}_config3.pkl', 'wb') as f:
        pickle.dump(config3_data, f)
    print(f"Config 3 saved: {A3.shape} adjacency matrix with {num_edges3} edges, {edge_index3.shape} edges in edge_index")

    print("All graph configurations saved to data/graph/")

Testing adjacency matrix construction:
Config 0 saved: (30, 30) adjacency matrix with 30 edges, torch.Size([2, 30]) edges in edge_index

Building graph with k=2
Config 1 saved: (30, 30) adjacency matrix with 60 edges, torch.Size([2, 60]) edges in edge_index
Config 2 saved: (30, 30) adjacency matrix with 57 edges, torch.Size([2, 57]) edges in edge_index
Config 3 saved: (30, 30) adjacency matrix with 30 edges, torch.Size([2, 30]) edges in edge_index
All graph configurations saved to data/graph/

Building graph with k=3
Config 1 saved: (30, 30) adjacency matrix with 90 edges, torch.Size([2, 90]) edges in edge_index
Config 2 saved: (30, 30) adjacency matrix with 84 edges, torch.Size([2, 84]) edges in edge_index
Config 3 saved: (30, 30) adjacency matrix with 43 edges, torch.Size([2, 43]) edges in edge_index
All graph configurations saved to data/graph/

Building graph with k=4
Config 1 saved: (30, 30) adjacency matrix with 120 edges, torch.Size([2, 120]) edges in edge_index
Config 2 saved: 